## Comparing Framewise displacement per condition

In [57]:
import numpy as np
import pandas as pd
import pingouin as pg
import glob

In [58]:
subject_list = ['029','040','1074','1205','1223','1237','1245','1247',
               '1280','1301','1337','1350','1359','1374','1392','020',
               '1072','1099','1206','1212','1216','1258','1266','1268',
               '1320','1340','1345','1346','1362','1379','1393','1457','1460',
               '1373','1272','1291','1309','1210','1218','1221','1303','1312','1326','1338']

In [59]:
# read FD
file_temp = '/media/Data/Lab_Projects/RCF/neuroimaging/RCF_Bids/derivatives/fmriprep/sub-{sub}/ses-1/func/sub-{sub}_ses-1_task-task*_desc-confounds_regressors.tsv'
events_temp = '/media/Data/Lab_Projects/RCF/neuroimaging/RCF_Bids/event_files/ses-1/sub-{sub}.csv'

In [106]:
df = []
for sub in subject_list:
    file = file_temp.format(sub=sub)
    file = glob.glob(file)[0]
    print(file)
    events = events_temp.format(sub=sub)
    fd = pd.read_csv(file, sep='\t').fillna(0)
    fd['framewise_displacement'].values
    # read events file
    ev = pd.read_csv(events)

    # average FD per condition
    fd_mean_us = []
    fd_mean_cs = []
    fd_mean_csm = []
    fd_mean_total = []
    f = fd['framewise_displacement'].values
    for i in ev.iterrows():
        on =i[1].onset
        cond = i[1].trial_type
       # print(cond)
        # average FD
        f_avg = np.mean(f[on:on+4])
        fd_mean_total.append(f_avg)   
        if (cond=='US_Aplus1' or cond=='US_Bplus1' or cond=='US_Aplus2' or cond=='US_Bplus2'):
          #  print('US')
            fd_mean_us.append(f_avg)
        elif (cond=='Aplus1' or cond=='Bplus1' or cond=='Aplus2' or cond=='Bplus2'):
           # print('Plus')
            fd_mean_cs.append(f_avg)
        else:
            #print('Minus')
            fd_mean_csm.append(f_avg)
    # conditions for each subject
    conditions = ['US','CS','CSm']
    # average per condition
    fd_s = [np.mean(fd_mean_us), np.mean(fd_mean_cs), np.mean(fd_mean_csm)]
    newD = pd.DataFrame({'subject': pd.Series([sub,sub,sub]), 'cond':pd.Series(conditions), 'fd':pd.Series(fd_s)})
    df.append(newD)

/media/Data/Lab_Projects/RCF/neuroimaging/RCF_Bids/derivatives/fmriprep/sub-029/ses-1/func/sub-029_ses-1_task-task556_desc-confounds_regressors.tsv
/media/Data/Lab_Projects/RCF/neuroimaging/RCF_Bids/derivatives/fmriprep/sub-040/ses-1/func/sub-040_ses-1_task-task556_desc-confounds_regressors.tsv
/media/Data/Lab_Projects/RCF/neuroimaging/RCF_Bids/derivatives/fmriprep/sub-1074/ses-1/func/sub-1074_ses-1_task-task505_desc-confounds_regressors.tsv
/media/Data/Lab_Projects/RCF/neuroimaging/RCF_Bids/derivatives/fmriprep/sub-1205/ses-1/func/sub-1205_ses-1_task-task505_desc-confounds_regressors.tsv
/media/Data/Lab_Projects/RCF/neuroimaging/RCF_Bids/derivatives/fmriprep/sub-1223/ses-1/func/sub-1223_ses-1_task-task505_desc-confounds_regressors.tsv
/media/Data/Lab_Projects/RCF/neuroimaging/RCF_Bids/derivatives/fmriprep/sub-1237/ses-1/func/sub-1237_ses-1_task-task505_desc-confounds_regressors.tsv
/media/Data/Lab_Projects/RCF/neuroimaging/RCF_Bids/derivatives/fmriprep/sub-1245/ses-1/func/sub-1245_ses

In [107]:
d = pd.concat(df)
d

,subject,cond,fd
0,029,US,0.178123
1,029,CS,0.195050
2,029,CSm,0.186389
0,040,US,0.195918
1,040,CS,0.196429
...,...,...,...
1,1326,CS,0.171364
2,1326,CSm,0.238781
0,1338,US,0.192520
1,1338,CS,0.191488


In [108]:
groupDF = pd.read_csv('groups.csv')
groupDF['subject'] = groupDF['subject'].astype(int).astype(str)
for i in groupDF.iterrows():
    # change two numbers to three
    if len(i[1].subject) < 4:
        groupDF.at[i[0],'subject'] = '0' + i[1].subject


groupDF.head()

,group,subject
0,PTSD,035
1,CC,011
2,PTSD,040
3,PTSD,029
4,CC,020


In [116]:
dg = pd.merge(d, groupDF)
dg

,subject,cond,fd,group
0,029,US,0.178123,PTSD
1,029,CS,0.195050,PTSD
2,029,CSm,0.186389,PTSD
3,040,US,0.195918,PTSD
4,040,CS,0.196429,PTSD
...,...,...,...,...
127,1326,CS,0.171364,CC
128,1326,CSm,0.238781,CC
129,1338,US,0.192520,PTSD
130,1338,CS,0.191488,PTSD


In [126]:
dg.to_csv('movement.csv', index=False)